In [1]:
import time
#
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split
from math import *
from sklearn import metrics
#
pd.set_option('display.max_columns', 500)  # force pandas to display all columns for better visual inspection
# plot plots inline
%matplotlib inline 

In [2]:
trn_all = pd.read_csv('../data/train.csv')  # column #0 in our file is index
# trn_1 = pd.read_csv('../data/train_1.csv', index_col=0)

In [3]:
#selected features and Cut off outliers of Expected >= 69
index=list(trn_all)
my_indices = [0,1,2,3,5,6,7,9,10,15,17, 18, 23]
trn_new = trn_all[[index[i] for i in my_indices]]

trn_new = trn_new[trn_new['Expected']<69]

In [4]:
#Cut off Ref values < 0
#trn_new[trn_new['Ref_5x5_50th']<0] = np.nan
#trn_new[trn_new['Ref_5x5_90th']<0] = np.nan
#trn_new[trn_new['RefComposite']<0] = np.nan
#trn_new[trn_new['RefComposite_5x5_50th']<0] = np.nan
#trn_new[trn_new['RefComposite_5x5_90th']<0] = np.nan
#trn_new[trn_new['Ref']<0] = np.nan

In [5]:
#combine observations with same ID by using mean
#replace Nan by overall mean
trn_mean = trn_new.groupby(trn_new.Id).mean()
index2 = list(trn_mean)
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)



trn_mean= pd.DataFrame(imp.fit_transform(trn_mean),index = trn_mean.index, columns=index2)

In [6]:
#train and test data preparation
X_trn = trn_mean.loc[:,'minutes_past':'Zdr_5x5_90th'].values
y_trn = np.log1p(trn_mean.loc[:,'Expected'].values)


# Split data as training and validation set
#[X_trn, X_test, y_trn, y_test] = train_test_split(X, Y, test_size = 0.3)


In [7]:
X_trn

array([[ 29.83333333,  10.        ,  21.40572927, ...,   0.59093184,
          0.22583356,   1.98369293],
       [ 29.08333333,   2.        ,  16.625     , ...,   0.38020833,
          0.41666667,   0.78125   ],
       [ 30.75      ,  10.        ,  21.40572927, ...,   0.59093184,
          0.22583356,   2.0625    ],
       ..., 
       [ 26.5       ,  14.        ,  21.40572927, ...,   0.59093184,
          0.22583356,   1.98369293],
       [ 30.76470588,  11.        ,  26.66666667, ...,   0.59093184,
          0.22583356,   1.98369293],
       [ 28.38461538,   9.        ,  28.16666667, ...,   0.875     ,
          0.5625    ,   3.02083333]])

In [8]:
clf = RandomForestRegressor(n_estimators=100,n_jobs=3)  # NOTE: n_jobs=-1 will use all of your cores, set to a prefered number

In [9]:
#train model
t = time.time()
clf.fit(X_trn, y_trn)
print time.time()-t

586.789396048


In [10]:
t

1447215451.404886

In [11]:
#test performance
#y_predict = clf.predict(X_test)
#score = metrics.mean_absolute_error(y_test, np.exp(y_predict)-1) #MAE

In [12]:
#print score

In [13]:
#generate test result
test = pd.read_csv('../data/test.csv')

#selected features and Cut off outliers of Expected >= 69
index=list(test)
my_indices = [0,1,2,3,5,6,7,9,10,15,17, 18]
test_new = test[[index[i] for i in my_indices]]
#Cut off Ref values < 0
#test_new[test_new['Ref_5x5_50th']<0] = np.nan
#test_new[test_new['Ref_5x5_90th']<0] = np.nan
#test_new[test_new['RefComposite']<0] = np.nan
#test_new[test_new['RefComposite_5x5_50th']<0] = np.nan
#test_new[test_new['RefComposite_5x5_90th']<0] = np.nan
#test_new[test_new['Ref']<0] = np.nan

#combine observations with same ID by using mean
#replace Nan by overall mean
test_mean = test_new.groupby(test_new.Id).mean()
index2 = list(test_mean)
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
test_mean= pd.DataFrame(imp.fit_transform(test_mean),index=test_mean.index,columns=index2)

test_X =test_mean.loc[:,'minutes_past':'Zdr_5x5_90th'].values

test_y_predict = np.exp(clf.predict(test_X))-1

In [14]:
#generate output file
#0.75 from prediction and 0.25 from marshall palmer
marshall = pd.read_csv('../data/MP_r_09.csv')

test_result_exist = pd.DataFrame()
test_result_exist['Id'] = test_mean.index
test_result_exist['Expected'] = test_y_predict

test_result = pd.DataFrame()
test_result['Id'] = test['Id'].unique()
test_result = pd.merge(test_result, test_result_exist, how='left', on=['Id'], sort=True)
test_result.loc[test_result['Expected'].isnull(), 'Expected'] = marshall.loc[test_result['Expected'].isnull(), 'Expected']
test_result.loc[test_result['Expected'].notnull(), 'Expected'] = 0.75*test_result.loc[test_result['Expected'].notnull(), 'Expected']+0.25*marshall.loc[test_result['Expected'].notnull(), 'Expected']

test_result.to_csv('../data/pks_result1.csv', index=False)

In [15]:
test_result

,Id,Expected
0,1,0.715270
1,2,1.404610
2,3,2.101069
3,4,3.329112
4,5,0.959228
5,6,1.340044
6,7,3.290369
7,8,2.161635
8,9,1.410020
9,10,6.215671
